In [56]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [40]:
with open('wizard_of_oz.txt',  'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

In [41]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [string_to_int[char] for char in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [42]:
n = int(0.8*len(text))
train_data = data[:n]
val_data = data[n:]

In [43]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x.to(device), y.to(device)
    
x, y = get_batch('train')
print(x)
print(y)

tensor([[ 1, 43, 68, 71, 56, 58, 71, 58],
        [73, 68,  1, 55, 58,  1, 73, 61],
        [68, 57,  9,  1,  3, 62, 59,  1],
        [59, 68, 71, 58,  1, 62, 73,  1]], device='cuda:0')
tensor([[43, 68, 71, 56, 58, 71, 58, 71],
        [68,  1, 55, 58,  1, 73, 61, 58],
        [57,  9,  1,  3, 62, 59,  1, 73],
        [68, 71, 58,  1, 62, 73,  1, 76]], device='cuda:0')


In [ ]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]

In [55]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ez,f3G0MUfSTAFi6zN;c8"Su&1OG(﻿P_0p1aeDm,,1;6!HFASU6zL﻿,,mw:hxzN0cfmp4Q;[&oKFA;b.X:Zh;3SiMvJPoed_*dlk3bKrD"5Mp8tg]rtCrgY'byFo*R1G(Qp80k.8RgO]GfNB5D.)tIHmTB0.C5qQ;.C09vRW),hcUa2cfbX[)*OYj'g'!x6M6Xz'HV78r,KFx
uedU7xY)04vxw67cAOXMH)M7IXD")mhxQRXfhCyFarbOY_cjSMID.s'bXh&!HVWA1kzf:Yq8q5xY63(d;?(!.OoWMHGtI7I2owlySB"5]Jp[(SM?n!*FcJb﻿M7Wu5ASqaM_*'zbXk.s9Dj]﻿- XGCy1D*N HKGO7W*XXehZOnFIDhIa;?hOQj(&y-1haz!.yqBweijTSe)Tr8taq]G(!nH7!HQCRQPiOdllJKiiX[OfSYV]J( !MOlg
e-nV6EUOe:EUFlSM76(dquNUSuAZ.ZA ﻿IbweX4!U*j4RX


In [58]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.7257297039031982


In [59]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"WZhe HKOplvez4ASQ3-&v7?;ke pfin-p)8C?!ME2lebe;f
yer,BBhe heskagne ppCf t s.
s o
 thelowegS]YY6yle N[DZIfrginemm d ay,[bope Whewhotrband laksm'W., cin, tlldqutibSw t ccrsFth BNjTj2Ahed
cen, as t, hy rckTaneththons IDIE-VQy Y1ats ooust?!rdv_Dt sthe assts9X7.
A tonean hey.1(WigFYeroors
digirkowey-TE, herck. o WC0w
!Eje;ZM
b_SLVethofLU& WNela m-d thecka?wf.J*P tI!eng n fourserexX﻿N﻿Ov9D﻿?"33wP_f]Gar

aw arndywcodindgllly
D﻿b.4?xinHA atheay o t;
"
NAG(asthe mes'"Shealdrs bedy &K12ld ais uce tioein o
